In [1]:
import pandas as pd

### Observaciones iniciales entre los tres datasets

In [2]:
df_applist = pd.read_csv("../data/app_list.csv")
print(f"app_list.csv tiene {df_applist.shape[0]} filas y {df_applist.shape[1]} columnas.")

app_list.csv tiene 29235 filas y 2 columnas.


In [3]:
df_steam = pd.read_csv("../data/steam_app_data.csv")
print(f"steam_app_data.csv tiene {df_steam.shape[0]} filas y {df_steam.shape[1]} columnas.")

steam_app_data.csv tiene 29235 filas y 39 columnas.


In [4]:
df_steamspy = pd.read_csv("../data/steamspy_data.csv")
print(f"steamspy_data.csv tiene {df_steamspy.shape[0]} filas y {df_steamspy.shape[1]} columnas.")

steamspy_data.csv tiene 29235 filas y 20 columnas.


Podemos observar que los tres datasets tienen el mismo número de columnas. No obstante, antes de realizar unirlos, analizaremos los tres datasets por separado.

### Dataset *app_list.csv*

#### Exploración

Columnas:
- appid: código del juego. (int)
- name: nombre del juego. (str)

In [5]:
df_applist.head()

,appid,name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force


Este dataframe es sencillo, puesto que solo existen dos columnas. A continuación miraremos si hay duplicados y/o valores nulos.

In [6]:
print(f"El dataframe tiene {df_applist['name'].duplicated().sum()} nombres repetidos.")

El dataframe tiene 77 nombres repetidos.


In [7]:
print(df_applist.isnull().sum())

appid    0
name     5
dtype: int64


#### Conclusiones

- Este es el dataset que une la información de los otros dos. No es aconsejable cambiarlo, pero sí tenerlo en cuenta para las exploraciones de los otros ficheros.
- Puesto que tiene solo dos columnas, el fichero podría ser un diccionario donde la clave es *appid* y el valor *name*. Este nuevo fichero tendría la extension *.json*. 

### Dataset *steam_app_data.csv*

#### Exploración

Columnas:
- type: tipo de producto. (str)
- name: nombre del videojuego. (str)
- steam_appid: código del videojuego. (int)
- required_age: edad mínima para jugar (int)
- is_free: ¿el videojuego es gratuito? En ningún caso se define el precio. (bool)
- controller_support: ¿se puede jugar al videojuego con mando? (bool) 
- dlc: contenido de videojuego descargable, si es que tiene. (list[int])
- detailed_description: descripción detallada del videojuego. (str)
- about_the_game: sinopsis del videojuego. (str)
- short_description: descripción detallada del videojuego. (str)
- fullgame: ¿el videojuego está completo? (bool)
- supported_languages: idiomas del videojuego. (list[str]) *
- header_image: URL de la imagen de encabezado en Steam. (str)
- website: página web oficial del videojuego. (str)
- pc_requirements: hardware y software necesario para jugar al videojuego en Windows. (dict) *
- mac_requirements: hardware y software necesario para jugar al videojuego en Mac. (dict) *
- linux_requirements: hardware y software necesario para jugar al videojuego en Linux. (dict) *
- legal_notice: copyright/derechos de autor. (str) *
- drm_notice: Digital Rights Management. (str) *
- ext_user_account_notice: otros sistemas de cuentas ajenos a Steam para iniciar sesion en el videojuego. (str)
- developers: desarrolladores del videojuego. (list[str])
- publishers: publicadores del videojuego. (list[str])
- demos: demos, si es que tiene. (dict)
- price_overview: precio del videojuego en GBP, teniendo en cuenta descuentos. (dict)
- packages: código de los paquetes de ofertas en los que esté el videojuego. (list)
- package_groups: paquete de ofertas en los que esté el videojuego. (list[dict])
- platforms: sistemas operativos en los que el videojuego está disponible. (dict)
- metacritic: puntuación de la página web *metacritic* junto con el enlace a la página correspondiente. (dict)
- reviews: comentarios de páginas web. (str) * 
- categories: categorías a las que pertenece el videojuego. (dict) **
- genres: géneros a los que pertenece el videojuego. (dict) **
- screenshots: capturas de pantalla del videojuego. (dict)
- movies: tráiler del juego. (dict)
- recommendations: total de recomendaciones por parte de los usuarios (dict)
- achievements: logros de Steam del videojuego. (dict)
- release_date: fecha de publicación en mes y año, también se distingue si se publicará en el futuro. (dict)
- support_info: página web y correo de soporte. (dict)
- background: imagen de fondo del videojuego en la página de Steam. (str)
- content_descriptors: información sobre contenido no apto para todos los públicos y notas al respecto. (dict)


In [8]:
pd.set_option('display.max_columns', None) # Puesto que el dataframe tiene muchas columnas, quitamos el límite de visualización de columnas 
df_steam.head(3)

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 719, 'final': 7...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 65735},{'total': 0},"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 399, 'final': 3...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 2802},{'total': 0},"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,game,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 399, 'final': 3...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 1992},{'total': 0},"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"


Miramos de qué tipo se consideran las columnas del dataframe.

In [9]:
df_steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29235 entries, 0 to 29234
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   type                     29086 non-null  object 
 1   name                     29234 non-null  object 
 2   steam_appid              29235 non-null  int64  
 3   required_age             29086 non-null  float64
 4   is_free                  29086 non-null  object 
 5   controller_support       5998 non-null   object 
 6   dlc                      4975 non-null   object 
 7   detailed_description     29060 non-null  object 
 8   about_the_game           29060 non-null  object 
 9   short_description        29060 non-null  object 
 10  fullgame                 0 non-null      float64
 11  supported_languages      29072 non-null  object 
 12  header_image             29086 non-null  object 
 13  website                  19252 non-null  object 
 14  pc_requirements       

36 de 39 columnas se consideran objetos.

**Observaciones para la limpieza**:

[inserir texto]

Antes de observar valores nulos, miramos si los nombres de los juegos se repiten.

In [10]:
print(f"El dataframe tiene {df_steam['name'].duplicated().sum()} nombres repetidos.")

El dataframe tiene 150 nombres repetidos.


Miramos los valores nulos del los datos

In [11]:
print(df_steam.isnull().sum())

type                         149
name                           1
steam_appid                    0
required_age                 149
is_free                      149
controller_support         23237
dlc                        24260
detailed_description         175
about_the_game               175
short_description            175
fullgame                   29235
supported_languages          163
header_image                 149
website                     9983
pc_requirements              149
mac_requirements             149
linux_requirements           149
legal_notice               19169
drm_notice                 29077
ext_user_account_notice    28723
developers                   264
publishers                   149
demos                      27096
price_overview              3712
packages                    3370
package_groups               149
platforms                    149
metacritic                 26254
reviews                    23330
categories                   714
genres    

Vemos que en la columna *name* hay un solo valor nulo. Con esto, 4 de 5 (o todos) casos de valores nulos del fichero *app_list.csv* se resuelven.
Observamos que en distintas columnas hay el mísmo número de nulos (149).
Sospechamos que hay 149 filas con los valores nulos en todas las columnas menos en *name* y *steam_appid*. Para comprobarlo, filtraremos de este modo:

In [12]:
no_nan = ["name", "steam_appid"] # columnas que sí que tendrán valores
cols = df_steam.columns.difference(no_nan) # selección de todas las columnas menos las que tendrán valores
mask = df_steam[cols].isna().all(axis=1) # de todo el dataframe, se filtraran las líneas con las condiciones estipuladas en el markdown anterior
result = df_steam[mask] # resultado del filtrado.
print(f"Se han detectado {result.shape[0]} casos donde todas las columnas menos name y steam_appid tienen valores nulos.")
result.head()

Se han detectado 149 casos donde todas las columnas menos name y steam_appid tienen valores nulos.


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
26,NaN,Half-Life: Opposing Force,852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,NaN,Half-Life: Opposing Force,4330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,NaN,Half-Life: Opposing Force,8740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,NaN,Half-Life: Opposing Force,8955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336,NaN,Half-Life: Opposing Force,11610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Efectivamente, la sospecha se confirma. Se puede ver que las primeras 5 filas del último output tienen el mismo nombre, lo cual significa que hay casos de nulos relacionados con los duplicados. A continuación veremos cuántos duplicados eliminamos si quitamos estas 149 filas.

In [13]:
df_clean = df_steam.loc[~df_steam.index.isin(result.index)]
print(f"El nuevo dataframe tiene {df_clean.shape[0]} filas.") # Tiene 149 filas menos que df_steam
print(f"El nuevo dataframe tiene {df_clean['name'].duplicated().sum()} nombres repetidos.")

El nuevo dataframe tiene 29086 filas.
El nuevo dataframe tiene 64 nombres repetidos.


Con esto afirmamos que no todos las filas llenas de valores nulos eran juegos duplicados. A continuación volveremos a ver los valores nulos por si podemos relacionar los duplicados con alguna columna.

In [14]:
print(df_clean.isnull().sum())

type                           0
name                           1
steam_appid                    0
required_age                   0
is_free                        0
controller_support         23088
dlc                        24111
detailed_description          26
about_the_game                26
short_description             26
fullgame                   29086
supported_languages           14
header_image                   0
website                     9834
pc_requirements                0
mac_requirements               0
linux_requirements             0
legal_notice               19020
drm_notice                 28928
ext_user_account_notice    28574
developers                   115
publishers                     0
demos                      26947
price_overview              3563
packages                    3221
package_groups                 0
platforms                      0
metacritic                 26105
reviews                    23181
categories                   565
genres    

#### Conclusiones

hay algunas columnas que contienen código HTML. Para ello, usaremos regex para quitar las etiquetas HTML
df["languages"] = df["languages"].str.replace(r"<[^>]+>|\*", "", regex=True).str.strip()


### Dataset *steamspy_data.csv*

#### Exploración

#### Conclusiones